In [1]:
import pandas as pd
import numpy as np
import nltk
import json

In [13]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [14]:
%run tag_to_filter.v1.ipynb

In [4]:
df_transitions = pd.read_csv(data_directory+'df_transitions.csv', 
                             sep='§', 
                             engine='python',
                             index_col=0, 
                             encoding='utf-8')

In [5]:
class Markov:
    def __init__(self, name, transitions):
        self.name = name
        self.transitions = transitions
        
    def predict(self, event):
        predict = predict_next_state(event, self.transitions)
        return predict
    
    def random_state(self):
        return np.random.choice(self.transitions.index)
    
markov = Markov('Markov', df_transitions)
markov.predict(markov.random_state())

'{"filters": {"manufacturer": ["boeing", "cirrus ", "pilatus"], "aircraft": ["zlin_z_242"], "company": ["blue1", "dragonair", "freebird_airlines", "tropic_air"], "country": [], "category": ["business", "premium"], "date": ["00072015", "19032012"]}, "tab": "food"}'

In [31]:
class Intent:
    def __init__(self, name):
        self.name = name

class Afficheur(Intent):
    def __init__(self, name, model_tagger, model_predicteur):
        self.name = name
        self.tagger = model_tagger
        self.predicteur = model_predicteur
    
    def get_tags(self, sentence):
        #request = self.tagger.predict([sentence])[0]
        request = [{'in': 'O'},{'Sweden': 'COUN_B'},{'France': 'COUN_B'},
                   {'from': 'O'},{'SPRING': 'DATE1_B'},{'1902': 'DATE1_B'},
                   {'to': 'O'},{'AUTUMN': 'DATE2_B'},{'2017': 'DATE2_E'},
                   {'cloud': 'STAT_B'},{'of': 'STAT_I'},{'words': 'STAT_E'},
                   {'for': 'O'},{'Pilatus': 'MANU_B'},{'Pilatus': 'O'}]
        return request
    
    def get_filters(self, sentence):
        request = self.get_tags(sentence)
        tags_values = extract_tags(request)
        filters = tag_to_filters(tags_values)
        filters = apply_date(filters)
        event = {
            CT_tabs : CT_tabs_default,
            CT_filt : filters,
        }
        return event
    
    def interact(self, sentence):
        event = self.get_filters(sentence)
        event_state = json.dumps(event)
        event_state_hash = json_string_to_hash(event_state)
        pred_state_hash = self.predicteur.predict(event_state_hash)
        pred_sentence = make_sentence_fom_json(json.loads(pred_state_hash))
        
        res = {"in_sate" : event_state_hash,
               "out_state" : pred_state_hash,
               "out_sentence" : pred_sentence,}
        
        return res
    
Afficheur_Intent = Afficheur(name='Afficheur', model_tagger='tagger', model_predicteur=markov)
Afficheur_Intent.name

'Afficheur'

In [32]:
class Classifieur:
    def __init__(self, name, model_classifieur):
        self.name = name
        self.classifieur = model_classifieur
        
    def predict(self, sentence):
        return 0

Classifieur = Classifieur(name='Intent classifieur', model_classifieur="")
Classifieur.predict(sentence="")

0

In [33]:
intents  = {"classifieur" : Classifieur,
            "intents" : {0 : Afficheur_Intent}
           }

intents

{'classifieur': <__main__.Classifieur at 0x7f04ade72550>,
 'intents': {0: <__main__.Afficheur at 0x7f04adfd6da0>}}

In [34]:
class ChatBot:
    def __init__(self, name, intents={}, classifieur=""):
        self.name = name
        self.classifieur = intents['classifieur']
        self.intents = intents['intents']
        
    def get_intent_classification(self, sentence):
        return self.classifieur.predict(sentence)
    
    def interact(self, sentence):
        intent_id = self.get_intent_classification(sentence)
        intent = self.intents[intent_id]
        interaction = intent.interact(sentence)
        return interaction

    
    
Hubert = ChatBot(name='Chatbot Hubert', intents=intents)
Hubert.get_intent_classification('aertetreb') 

0

In [42]:
res = Hubert.interact('aertetreb')
for k,v in res.items():
    print(k,':\t',v,'\n')

in_sate :	 {"filters": {"manufacturer": ["pilatus"], "aircraft": [], "company": [], "country": ["france", "sweden"], "category": [], "date": ["01031902", "30112017"]}, "tab": "default"} 

out_state :	 {"filters": {"manufacturer": ["pilatus"], "aircraft": ["sw_4"], "company": ["air_slovakia", "onur_air", "swaziland_airlink", "viking_airlines"], "country": [], "category": [], "date": ["03052011", "06112011"]}, "tab": "default"} 

out_sentence :	 We suggest you the global study from 03-05-2011 to 06-11-2011 for the manufacturerpilatus and for the aircraftsw_4 and for the companies air_slovakia,onur_air,swaziland_airlink,viking_airlines and. If you agree, click onto the following link ;) 

